In [14]:
import pandas as pd
import numpy as np
import re
import datetime
import os
pd.options.mode.chained_assignment = None  # default='warn'
import importlib
import global_settings
from unidecode import unidecode
importlib.reload(global_settings)


<module 'global_settings' from 'D:\\HieuNV\\HMS_MH\\global_settings.py'>

In [15]:
setts = global_settings.Settings()

Directory found: D:\HieuNV\HMS_MH\input
Directory found: D:\HieuNV\HMS_MH\input\tmp1
Directory found: D:\HieuNV\HMS_MH\input\tmp2
Directory found: D:\HieuNV\HMS_MH\output


In [16]:
mh = pd.read_csv(setts.input_folder + '/MyHonda_20241220.csv', dtype=str)
mh.columns = setts.clean_columns(mh)
mh = mh.rename(
    columns = { 'customer_id' : 'customer_code',
                'name' : 'customer_name',
                'phone' : 'mobile_phone',
                'address' : 'customer_address'}
)
mh

,customer_code,vin,customer_name,mobile_phone,customer_address,district_id,province_id,deleted_at
0,100000,RLHJF7936KZ008871,NaN,0384317686,NaN,NaN,NaN,NaN
1,10000002,RLHJK0357PY141505,Lê Văn Hảo,0974846798,dieu sơn - yên lâm,285,38,NaN
2,10000007,RLHJK1214PZ653396,Lục Thị Nguyệt,0969601838,"khuổi mý , trần phú",62,6,NaN
3,10000009,RLHJK140XPZ289411,NaN,0988102449,NaN,NaN,NaN,NaN
4,10000010,RLHJK0346RY300475,Nguyễn Thu Hương,0986986488,"số 3, Ngách 7/6 Nguyên Xá, Nguyên Xá 3, Bắc Từ...",15,1,NaN
...,...,...,...,...,...,...,...,...
8543152,11438988,RLHJK0352RY189318,Trần Thị Thuỳ Linh,363900845,106/19 phan văn hân,561,79,NaN
8543153,11438991,RLHJK1918RZ333443,NaN,907554880,NaN,NaN,NaN,NaN
8543154,11439002,RLHJK0353RY175380,Phạm Thị Minh Trang,974768750,Tổ 22 cụm 3 Nhật Tân - Tây Hồ - Hà Nội,3,1,NaN
8543155,11439016,RLHJK1409RZ410416,NaN,866774688,NaN,NaN,NaN,NaN


In [17]:
# mh = mh.dropna(subset=['customer_name', 'mobile_phone', 'customer_address'], how='all')
mh = mh.drop_duplicates(subset=['customer_code','customer_name','mobile_phone','customer_address','vin'])
print(len(mh))

8541893


In [18]:
mh = mh.dropna(subset=['mobile_phone'], how='all')
mh['mobile_phone'] = mh['mobile_phone'].apply(lambda x: setts.clean_phone(x))

mh['province_id'] = mh['province_id'].astype(str).apply(lambda x: x.zfill(2))
mh['district_id'] = mh['district_id'].astype(str).apply(lambda x: x.zfill(3))
# Map district
district_map = pd.read_csv(setts.input_folder + '/district.csv', dtype=str)[['code', 'name']].rename(columns={'name': 'district_name', 'code' : 'district_id'})
mh = pd.merge(left=mh, right=district_map, how='left', on='district_id')

# Map province
province_map = pd.read_csv(setts.input_folder + '/province.csv', dtype=str)[['code', 'name']].rename(columns={'name': 'city', 'code' : 'province_id'})
mh = pd.merge(left=mh, right=province_map, how='left', on='province_id')

In [19]:
mh = mh.fillna('')

mh['customer_address'] = mh['customer_address'] + ' ' + mh['district_name'] + ' ' +  mh['city']
mh['customer_address'] = mh['customer_address'].str.strip()
mh['customer_name'] = mh['customer_name'].apply(lambda x: setts.clean_name(x))
mh['customer_address'] = mh['customer_address'].apply(lambda x: setts.clean_address(x))

mh = mh[mh['mobile_phone'].apply(lambda x: len(str(x).strip()) > 0)]


mh = mh[['customer_code', 'customer_name', 'mobile_phone', 'customer_address', 'vin', 'city']]
mh

,customer_code,customer_name,mobile_phone,customer_address,vin,city
0,100000,,0384317686,,RLHJF7936KZ008871,
1,10000002,Lê Văn Hảo,0974846798,DIEU SƠN YÊN LÂM THANH HÓA,RLHJK0357PY141505,Thanh Hóa
2,10000007,Lục Thị Nguyệt,0969601838,KHUỔI MÝ TRẦN PHÚ NGÂN SƠN BẮC KẠN,RLHJK1214PZ653396,Bắc Kạn
3,10000009,,0988102449,,RLHJK140XPZ289411,
4,10000010,Nguyễn Thu Hương,0986986488,SỐ 3 NGÁCH 7 6 NGUYÊN XÁ NGUYÊN XÁ 3 BẮC TỪ LI...,RLHJK0346RY300475,Hà Nội
...,...,...,...,...,...,...
8541726,11438988,Trần Thị Thuỳ Linh,0363900845,106 19 PHAN VĂN HÂN SÔNG HINH HỒ CHÍ MINH,RLHJK0352RY189318,Hồ Chí Minh
8541727,11438991,,0907554880,,RLHJK1918RZ333443,
8541728,11439002,Phạm Thị Minh Trang,0974768750,TỔ 22 CỤM 3 NHẬT TÂN TÂY HỒ HÀ NỘI TÂY HỒ HÀ NỘI,RLHJK0353RY175380,Hà Nội
8541729,11439016,,0866774688,,RLHJK1409RZ410416,


In [20]:
mh['is_company'] = mh['customer_name'].fillna('').apply(lambda x: int(setts.mark_as_company(unidecode(x.lower()))))
mh['is_company'].value_counts()

0    8514918
1      11281
Name: is_company, dtype: int64

In [21]:
mh[mh['is_company'] == 1] 

,customer_code,customer_name,mobile_phone,customer_address,vin,city,is_company
25,1000014,CTY TNHH PHỤ TÙNG ĐỨC PHÁT,0986290660,AN DƯƠNG VƯƠNG,MRHRW1870LP081566,,1
2988,10000634,CTY TNHH MAXWELL ELEC TECH,0364008658,LONG BÌNH ĐỒNG NAI,RLHJK0341PZ499281,Đồng Nai,1
3647,10000733,CTy Cổ Phần Làng Tre Mũi Né,0368113899,,RLHJK1211PZ641464,,1
3660,10006942,Công Ty TNHH XNK Supkee,0707298278,45 ĐƯỜNG SỐ 34 P BÌNH TRỊ ĐÔNG B Q BÌNH TÂN TP...,RLHJK1404PZ034172,Hồ Chí Minh,1
3661,10006942,Công Ty TNHH XNK Supkee,0707298278,45 ĐƯỜNG SỐ 34 P BÌNH TRỊ ĐÔNG B Q BÌNH TÂN TP...,RLHJF6316HZ048829,Hồ Chí Minh,1
...,...,...,...,...,...,...,...
8539434,11437346,Nguyễn Ngân Hằng,0344728638,TỔ 5B KHU 2 QUẢNG NINH,RLHJK034XRZ141227,Quảng Ninh,1
8539511,9997179,Công Ty TNHH Thắng Thuỷ,0969199339,HÀ NỘI,RLHJA3937RY007672,Hà Nội,1
8540103,11438064,CÔNG TY CƠ KHÍ,0985770377,,RLHJA3939RY815036,,1
8540350,11438301,cty tnhh nghệ sĩ quyền linh,0931005510,,RLHKF5301RZ601863,,1


In [22]:
# global_settings.replace_city(mh,'city', 'city')
# mh

In [23]:
mh[:5000000].to_csv(setts.tmp1_folder + '/cleaned_mh_5M_1H.csv', index=False)
mh[5000000:].to_csv(setts.tmp1_folder + '/cleaned_mh_5M_2H.csv', index=False)


In [24]:
def count_blank_values(df, columns, column_combinations):
    blank_count = {}
    
    # Count blank values and calculate percentage for individual columns
    total_rows = len(df)
    for col in columns:
        blank_values = df[col].apply(lambda x: len(str(x).strip()) == 0).sum()
        blank_count[col] = {
            'count': blank_values,
            'percentage': (blank_values / total_rows) * 100 if total_rows > 0 else 0
        }
    
    # Count blank values and calculate percentage for specified combinations of columns
    for comb in column_combinations:
        combined_col_name = '&'.join(comb)
        blank_values = df[list(comb)].apply(lambda x: all(len(str(val).strip()) == 0 for val in x), axis=1).sum()
        blank_count[combined_col_name] = {
            'count': blank_values,
            'percentage': (blank_values / total_rows) * 100 if total_rows > 0 else 0
        }
        
    return blank_count

count_blank_values(mh, ['customer_name','mobile_phone','customer_address','vin'],
                      [['customer_name', 'customer_address']])

{'customer_name': {'count': 3823551, 'percentage': 44.84473092875266},
 'mobile_phone': {'count': 0, 'percentage': 0.0},
 'customer_address': {'count': 5199405, 'percentage': 60.98151122205804},
 'vin': {'count': 0, 'percentage': 0.0},
 'customer_name&customer_address': {'count': 3819084,
  'percentage': 44.7923394703783}}

In [25]:
def count_non_blank_values(df, columns, column_combinations):
    non_blank_count = {}
    
    # Count non-blank values and calculate percentage for individual columns
    total_rows = len(df)
    for col in columns:
        non_blank_values = df[col].apply(lambda x: len(str(x).strip()) > 0).sum()
        non_blank_count[col] = {
            'count': non_blank_values,
            'percentage': (non_blank_values / total_rows) * 100 if total_rows > 0 else 0
        }
    
    # Count non-blank values and calculate percentage for specified combinations of columns
    for comb in column_combinations:
        combined_col_name = '&'.join(comb)
        non_blank_values = df[list(comb)].apply(lambda x: all(len(str(val).strip()) > 0 for val in x), axis=1).sum()
        non_blank_count[combined_col_name] = {
            'count': non_blank_values,
            'percentage': (non_blank_values / total_rows) * 100 if total_rows > 0 else 0
        }
        
    return non_blank_count

In [26]:
count_non_blank_values(mh, ['customer_name','mobile_phone','customer_address','vin'],
                      [['customer_name', 'customer_address']])

{'customer_name': {'count': 4702648, 'percentage': 55.15526907124734},
 'mobile_phone': {'count': 8526199, 'percentage': 100.0},
 'customer_address': {'count': 3326794, 'percentage': 39.01848877794196},
 'vin': {'count': 8526199, 'percentage': 100.0},
 'customer_name&customer_address': {'count': 3322327,
  'percentage': 38.9660973195676}}